In [1]:
#!/usr/bin/env python

import os
import sys

import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
import tensorflow.compat.v1 as tf

from six.moves import urllib
from sklearn.model_selection import train_test_split

In [73]:
## Load Dataset

path = "/home/petra/tfx/data/wine-quality.csv"

dataset = pd.read_csv(path)

#labels = dataset['quality']

x_train, x_test = train_test_split(dataset, train_size=0.7)
y_train = x_train.pop('quality')
y_test = x_test.pop('quality')

In [79]:
y_train

589     6
1591    6
16      6
2973    6
3799    5
       ..
4688    6
4809    6
3354    7
13      7
3143    5
Name: quality, Length: 3428, dtype: int64

In [75]:
NUMERIC_COLUMNS = ['alcohol', 'chlorides', 'citric_acid', 'density', 'fixed_acidity',
                   'free_sulfur_dioxide', 'pH', 'residual_sugar', 'sulphates',
                   'total_sulfur_dioxide', 'volatile_acidity']

LABEL = 'quality'

feature_columns = []
for feature_name in NUMERIC_COLUMNS:
    feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

In [76]:
for _ in feature_columns:
    print(_)

NumericColumn(key='alcohol', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)
NumericColumn(key='chlorides', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)
NumericColumn(key='citric_acid', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)
NumericColumn(key='density', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)
NumericColumn(key='fixed_acidity', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)
NumericColumn(key='free_sulfur_dioxide', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)
NumericColumn(key='pH', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)
NumericColumn(key='residual_sugar', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)
NumericColumn(key='sulphates', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)
NumericColumn(key='total_sulfur_dioxide', shape=(1,), default_value=None, dtype=t

In [77]:
def make_input_fn(data_df, label_df, num_epochs=None, shuffle=True, batch_size=1000):
  def input_function():
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
    if shuffle:
      ds = ds.shuffle(batch_size)
    ds = ds.batch(batch_size).repeat(num_epochs)
    return ds
  return input_function

train_input_fn = make_input_fn(x_train, y_train, batch_size=len(x_train))
eval_input_fn = make_input_fn(x_test, y_test, num_epochs=1, shuffle=False, batch_size=len(x_test))


In [89]:
# Since data fits into memory, use entire dataset per layer. It will be faster.
# Above one batch is defined as the entire dataset.
n_batches = 1
est = tf.estimator.BoostedTreesRegressor(feature_columns,
                                          n_batches_per_layer=n_batches,
                                          n_trees=50, 
                                          max_depth=7,
                                          learning_rate=0.02
                                         )

est.train(train_input_fn, max_steps=500)



INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpbsw_mcyi', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.


In [71]:
# Eval.
result = est.evaluate(eval_input_fn)
#clear_output()
print(pd.Series(result).to_frame())

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-01-07T23:34:05Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpbhzs76_q/model.ckpt-350
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.16481s
INFO:tensorflow:Finished evaluation at 2021-01-07-23:34:05
INFO:tensorflow:Saving dict for global step 350: average_loss = 0.43126994, global_step = 350, label/mean = 5.869748, loss = 0.43126994, prediction/mean = 5.884742
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 350: /tmp/tmpbhzs76_q/model.ckpt-350
                          0
average_loss       0.431270
label/mean         5.869748
loss               0.431270
prediction/

In [87]:
#feature_columns.pop("VocabularyListCategoricalColumn")
#print("--> FEATURE_COLUMNS = " + str(feature_columns))

def serving_input_receiver_fn():
    inputs = {"x": tf.placeholder(shape=[None, 11], dtype=tf.float32)}
    return tf.estimator.export.ServingInputReceiver(inputs, inputs)

print("--> serving_input_receiver_fn = " + str(serving_input_receiver_fn))

est.export_saved_model(export_dir_base="/home/petra/tfx/data/model", serving_input_receiver_fn=serving_input_receiver_fn)

--> serving_input_receiver_fn = <function serving_input_receiver_fn at 0x7f76c4414290>


AttributeError: module 'tensorflow' has no attribute 'placeholder'

In [ ]:
def _example_serving_receiver_fn(tf_transform_graph, schema):
  """Build the serving in inputs.
  
  Args:
    tf_transform_graph: A TFTransformOutput.
    schema: the schema of the input data.
    
  Returns:
    Tensorflow graph which parses examples, applying tf-transform to them.
  """

  # Get feature specifications and remove label
  raw_feature_spec = _get_raw_feature_spec(schema)
  raw_feature_spec.pop(LABEL_KEY)
  
  # Creates a serving_input_receiver_fn that expects a serialized tf.Example
  # fed into a string placeholder. The function parses the tf.Example according
  # to the provided feature_spec, and returns all parsed Tensors as features.
  raw_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(
      raw_feature_spec, default_batch_size=None)
  serving_input_receiver = raw_input_fn()
    
      
  # Transform the features according to the transform_graph
  transformed_features = tf_transform_graph.transform_raw_features(
      serving_input_receiver.features)

  return tf.estimator.export.ServingInputReceiver(
      transformed_features, serving_input_receiver.receiver_tensors)

In [85]:
def _make_serving_input_fn(tf_transform_output):
  """Creates an input function reading from raw data.

  Args:
    tf_transform_output: Wrapper around output of tf.Transform.

  Returns:
    The serving input function.
  """
  raw_feature_spec = RAW_DATA_FEATURE_SPEC.copy()
  # Remove label since it is not available during serving.
  raw_feature_spec.pop(LABEL_KEY)

  def serving_input_fn():
    """Input function for serving."""
    # Get raw features by generating the basic serving input_fn and calling it.
    # Here we generate an input_fn that expects a parsed Example proto to be fed
    # to the model at serving time.  See also
    # tf.estimator.export.build_raw_serving_input_receiver_fn.
    raw_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(
        raw_feature_spec, default_batch_size=None)
    serving_input_receiver = raw_input_fn()

    # Apply the transform function that was used to generate the materialized
    # data.
    raw_features = serving_input_receiver.features
    transformed_features = tf_transform_output.transform_raw_features(
        raw_features)

    return tf.estimator.export.ServingInputReceiver(
        transformed_features, serving_input_receiver.receiver_tensors)

  return serving_input_fn

In [86]:
# Export the model.
serving_input_fn = _make_serving_input_fn(tf_transform_output)
exported_model_dir = os.path.join(working_dir, EXPORTED_MODEL_DIR)
estimator.export_saved_model(exported_model_dir, serving_input_fn)

NameError: name 'tf_transform_output' is not defined